##### The purpose of this notebook is to update OSDS inventory with sewer billing, private WWSA, and community data:
Script functionality:
The script has two steps of editing the inventory from different data collection methods

Takes 2025 OSDS inventory, marks OSDS that have sewer bill, sewer infrastructure, or in private wastewater service area to flag for remove

General steps:
1. Add new fields to OSDS (Lat, SewerBill, Flag)
2. Normalize TMKs in billing data and remove rows that are not in use 
3. Filter these subsets to only the **new values** that don't already exist in the AGOL tables
4. **Append** the new values to the AGOL tables
5. Calculate totals and final month usage for each village


#### Add new fields to OSDS shapefiles
##### Lat, SewBill, Flag, Priv will be added as Text fields

In [19]:
import arcpy, os
arcpy.env.overwriteOutput = True


In [1]:
FIELDS = ["Lat", "SewBill", "Flag", "Priv"]  # all TEXT fields

def add_text_fields(in_fc, names, length=16):
    existing = {f.name.lower() for f in arcpy.ListFields(in_fc)}
    for n in names:
        if n.lower() not in existing:
            arcpy.management.AddField(in_fc, n, "TEXT", field_length=length)
            print(f"{os.path.basename(in_fc)}: added {n} (TEXT,{length}).")
        else:
            print(f"{os.path.basename(in_fc)}: {n} exists, skipped.")

# Explicit list of shapefiles (add as many as you need)
shp_list = [
    r"C:\Users\aimee\Documents\SewerBillingCleanup\Downloaded_data\NEW_OSDS\NewOSDSv2\Kauai_OSDSv2\Kauai\Kauai_2023.shp",
    r"C:\Users\aimee\Documents\SewerBillingCleanup\Downloaded_data\NEW_OSDS\NewOSDSv2\Oahu_OSDSv2\Oahu\Oahu_OSDS_2023.shp",
    r"C:\Users\aimee\Documents\SewerBillingCleanup\Downloaded_data\NEW_OSDS\NewOSDSv2\MauiNuiv2\MauiNui\MauiNui_OSDS_2023.shp",
    r"C:\Users\aimee\Documents\SewerBillingCleanup\Downloaded_data\NEW_OSDS\NewOSDSv2\Hawaii_OSDSv2\Hawaii\Hawaii_OSDS_2023.shp",
]

for shp in shp_list:
    add_text_fields(shp, FIELDS, length=16)


Kauai_2023.shp: Lat exists, skipped.
Kauai_2023.shp: SewBill exists, skipped.
Kauai_2023.shp: Flag exists, skipped.
Kauai_2023.shp: added Priv (TEXT,16).
Oahu_OSDS_2023.shp: Lat exists, skipped.
Oahu_OSDS_2023.shp: SewBill exists, skipped.
Oahu_OSDS_2023.shp: Flag exists, skipped.
Oahu_OSDS_2023.shp: added Priv (TEXT,16).
MauiNui_OSDS_2023.shp: Lat exists, skipped.
MauiNui_OSDS_2023.shp: SewBill exists, skipped.
MauiNui_OSDS_2023.shp: Flag exists, skipped.
MauiNui_OSDS_2023.shp: added Priv (TEXT,16).
Hawaii_OSDS_2023.shp: Lat exists, skipped.
Hawaii_OSDS_2023.shp: SewBill exists, skipped.
Hawaii_OSDS_2023.shp: Flag exists, skipped.
Hawaii_OSDS_2023.shp: added Priv (TEXT,16).


#### Normalize TMKs in Billing Data

In [9]:
import arcpy

TABLE     = r"in_memory\billing_tbl"  # same as TMP_TBL from your stats cell
SRC_TMK   = "CleanTMK"
NEW_FIELD = "CleanTMK_long"

# add field if missing
if not any(f.name.lower() == NEW_FIELD.lower() for f in arcpy.ListFields(TABLE)):
    arcpy.management.AddField(TABLE, NEW_FIELD, "LONG")

# keep only digits, convert to LONG (blanks -> NULL)
arcpy.management.CalculateField(
    TABLE, NEW_FIELD, f"to_long(!{SRC_TMK}!)", "PYTHON3",
    "def to_long(v):\n"
    "    s = '' if v is None else str(v)\n"
    "    d = ''.join(c for c in s if c.isdigit())\n"
    "    return int(d) if d else None\n"
)

print("Cleaned TMK -> TMK_long.")


Cleaned TMK -> TMK_long.


#### Summarize Billing Data

##### Data was provided by each county seperately 
##### Rows that did not receieve sewer were deleted manually 

In [10]:
# Summarize Billing for Oahu, Maui, Kauai 

# INPUT / OUTPUT
IN_CSV  = r"C:\Users\aimee\Documents\SewerBillingCleanup\Downloaded_data\SewerBillingData\Clean_Mau_Kau_HNL_Billing.csv"
OUT_CSV = r"C:\Users\aimee\Documents\SewerBillingCleanup\Downloaded_data\SewerBillingData\Sum_Mau_Kau_HNL_Billing.csv"

# Temp in-memory names
TMP_TBL    = r"in_memory\billing_tbl"
STATS_TBL  = r"in_memory\billing_stats"
BILL_TMK   = "CleanTMK"                     # TMK field in the CSV
COUNT_FLD  = f"COUNT_{BILL_TMK}"            # Statistics output field name

# Clean any leftovers
for p in (TMP_TBL, STATS_TBL):
    if arcpy.Exists(p):
        arcpy.management.Delete(p)

# 1) CSV -> in_memory table (adds OID so tools can run)
arcpy.conversion.TableToTable(IN_CSV, r"in_memory", os.path.basename(TMP_TBL))

# 2) Statistics: COUNT of CleanTMK, grouped by CleanTMK
arcpy.analysis.Statistics(
    in_table=TMP_TBL,
    out_table=STATS_TBL,
    statistics_fields=[[BILL_TMK, "COUNT"]],
    case_field=BILL_TMK
)

# 3) Write stats table to CSV
fields = [BILL_TMK, COUNT_FLD]
with open(OUT_CSV, "w", newline="", encoding="utf-8") as f:
    w = csv.writer(f)
    w.writerow(fields)
    for row in arcpy.da.SearchCursor(STATS_TBL, fields):
        w.writerow(row)

print(f"Wrote stats CSV: {OUT_CSV}")


Wrote stats CSV: C:\Users\aimee\Documents\SewerBillingCleanup\Downloaded_data\SewerBillingData\Sum_Mau_Kau_HNL_Billing.csv


#### Join Oahu Laterals to Oahu OSDS 
##### Manually remove inactive, abandoned and removed laterals from dataset 

In [3]:

osds_points_path     = r"C:\Users\aimee\Documents\SewerBillingCleanup\Downloaded_data\NEW_OSDS\NewOSDSv2\Oahu_OSDSv2\Oahu\Oahu_OSDS_2023.shp"
laterals_path        = r"C:\Users\aimee\Documents\SewerBillingCleanup\Downloaded_data\SewerLines\SewerLateral_Oahu_Project\SewerLateral_Oahu_Project.shp"
osds_tmk_long_field  = "TMK"    # TMK LONG field in OSDS
laterals_tmk_long_field = "TMKLong" # TMK LONG field in laterals

# 1) Count laterals per TMK
lat_counts_table = r"in_memory\oahu_lateral_counts"
if arcpy.Exists(lat_counts_table):
    arcpy.management.Delete(lat_counts_table)

arcpy.analysis.Statistics(
    in_table=laterals_path,
    out_table=lat_counts_table,
    statistics_fields=[[laterals_tmk_long_field, "COUNT"]],
    case_field=laterals_tmk_long_field
)

# 2) Normalize the count field to a short, stable name: LatCount
count_src = next(f.name for f in arcpy.ListFields(lat_counts_table) if f.name.upper().startswith("COUNT"))
if not any(f.name.lower() == "latcount" for f in arcpy.ListFields(lat_counts_table)):
    arcpy.management.AddField(lat_counts_table, "LatCount", "LONG")
arcpy.management.CalculateField(lat_counts_table, "LatCount", f"!{count_src}!", "PYTHON3")

# 3) If OSDS already has LatCount from a previous run, drop it to avoid join collision
if any(f.name.lower() == "latcount" for f in arcpy.ListFields(osds_points_path)):
    arcpy.management.DeleteField(osds_points_path, ["LatCount"])

# 4) Join LatCount onto OSDS by TMK
arcpy.management.JoinField(
    in_data=osds_points_path,
    in_field=osds_tmk_long_field,
    join_table=lat_counts_table,
    join_field=laterals_tmk_long_field,
    fields=["LatCount"]
)

# 5) Ensure Lat field exists and set Y/N based on LatCount
if not any(f.name.lower() == "lat" for f in arcpy.ListFields(osds_points_path)):
    arcpy.management.AddField(osds_points_path, "Lat", "TEXT", field_length=16)

arcpy.management.CalculateField(osds_points_path, "Lat", "'N'", "PYTHON3")

osds_layer = "osds_oahu_layer"
arcpy.management.MakeFeatureLayer(osds_points_path, osds_layer)
fld = arcpy.AddFieldDelimiters(osds_points_path, "LatCount")
arcpy.management.SelectLayerByAttribute(osds_layer, "NEW_SELECTION", f"{fld} > 0")
if int(arcpy.management.GetCount(osds_layer)[0]) > 0:
    arcpy.management.CalculateField(osds_layer, "Lat", "'Y'", "PYTHON3")
arcpy.management.Delete(osds_layer)

print("Done: joined LatCount; Lat set to Y/N.")


Done: joined LatCount; Lat set to Y/N.


#### Join Kauai Laterals to Kauai OSDS 
##### Manually remove inactive, abandoned and removed laterals from dataset 

In [4]:

kauai_osds_path            = r"C:\Users\aimee\Documents\SewerBillingCleanup\Downloaded_data\NEW_OSDS\NewOSDSv2\Kauai_OSDSv2\Kauai\Kauai_2023.shp"
kauai_laterals_path        = r"C:\Users\aimee\Documents\SewerBillingCleanup\Downloaded_data\SewerLines\CountyofKauaiLateral_Project\CountyofKauaiLateral_Project.shp"    # Kauai sewer laterals (lines/points)
osds_tmk_long_field        = "TMK"        # LONG TMK field in OSDS (≤10 chars for .shp)
laterals_tmk_long_field    = "TMK"        # LONG TMK field in laterals (≤10 chars for .shp)

# 1) Count laterals per TMK
lat_counts_tbl = r"in_memory\kauai_lat_counts"
if arcpy.Exists(lat_counts_tbl):
    arcpy.management.Delete(lat_counts_tbl)

arcpy.analysis.Statistics(
    in_table=kauai_laterals_path,
    out_table=lat_counts_tbl,
    statistics_fields=[[laterals_tmk_long_field, "COUNT"]],
    case_field=laterals_tmk_long_field
)

# 2) Create a short, stable count field name to avoid truncation issues
count_src = next(f.name for f in arcpy.ListFields(lat_counts_tbl) if f.name.upper().startswith("COUNT"))
if not any(f.name.lower() == "LatCount".lower() for f in arcpy.ListFields(lat_counts_tbl)):
    arcpy.management.AddField(lat_counts_tbl, "LatCount", "LONG")
arcpy.management.CalculateField(lat_counts_tbl, "LatCount", f"!{count_src}!", "PYTHON3")

# 3) If OSDS already has LatCount from a prior run, drop it to avoid JoinField collisions
if any(f.name.lower() == "LatCount".lower() for f in arcpy.ListFields(kauai_osds_path)):
    arcpy.management.DeleteField(kauai_osds_path, ["LatCount"])

# 4) Join LatCount to OSDS by TMK (fields can have different names on each layer)
arcpy.management.JoinField(
    in_data=kauai_osds_path,
    in_field=osds_tmk_long_field,
    join_table=lat_counts_tbl,
    join_field=laterals_tmk_long_field,
    fields=["LatCount"]
)

# 5) Ensure OSDS has 'Lat' (TEXT), then set Y/N based on LatCount
if not any(f.name.lower() == "lat" for f in arcpy.ListFields(kauai_osds_path)):
    arcpy.management.AddField(kauai_osds_path, "Lat", "TEXT", field_length=16)

# Default all to 'N'
arcpy.management.CalculateField(kauai_osds_path, "Lat", "'N'", "PYTHON3")

# Flip to 'Y' where LatCount > 0
osds_layer = "kauai_osds_layer"
arcpy.management.MakeFeatureLayer(kauai_osds_path, osds_layer)
latcount_delim = arcpy.AddFieldDelimiters(kauai_osds_path, "LatCount")
arcpy.management.SelectLayerByAttribute(osds_layer, "NEW_SELECTION", f"{latcount_delim} > 0")
selected = int(arcpy.management.GetCount(osds_layer)[0])
if selected > 0:
    arcpy.management.CalculateField(osds_layer, "Lat", "'Y'", "PYTHON3")
arcpy.management.Delete(osds_layer)

# (Optional) remove the joined LatCount to keep OSDS clean for re-runs
if any(f.name.lower() == "LatCount".lower() for f in arcpy.ListFields(kauai_osds_path)):
    arcpy.management.DeleteField(kauai_osds_path, ["LatCount"])

print(f"Done: Kauai laterals joined by TMK; set Lat='Y' for {selected} feature(s).")


Done: Kauai laterals joined by TMK; set Lat='Y' for 21 feature(s).


#### Intersect Maui Laterals and Gravity Mains with TMK Parcels 
##### Join Maui laterals with gravity main due to lack of laterals. Buffer gravity main to touch TMK parcels
##### Then intersect OSDS with parcels that contain sewer infrastructure

In [5]:


# === EDIT THESE PATHS ===
maui_osds_points   = r"C:\Users\aimee\Documents\SewerBillingCleanup\Downloaded_data\NEW_OSDS\NewOSDSv2\MauiNuiv2\MauiNui\MauiNui_OSDS_2023.shp"           # OSDS points (Maui)
maui_parcels       = r"C:\Users\aimee\Documents\SewerBillingCleanup\Projected_data\tmk_state_projected\tmk_state_projected.shp"        # TMK parcels (polygons)
maui_laterals      = r"C:\Users\aimee\Documents\SewerBillingCleanup\Downloaded_data\SewerLines\MauiSewer_Lateral_Lines_Project\MauiSewer_Lateral_Lines_Project.shp"       # laterals (lines) — optional, used WITHOUT buffer
maui_gravity_mains = r"C:\Users\aimee\Documents\SewerBillingCleanup\Downloaded_data\SewerLines\MauiSewer_Gravity_Mains_Project\MauiSewer_Gravity_Mains_Project.shp"    # gravity mains (lines)

main_buffer = "13 Meters"    # buffer only the gravity mains

# 1) Buffer gravity mains (only)
mains_buf = r"in_memory\mains_buf"
if arcpy.Exists(mains_buf):
    arcpy.management.Delete(mains_buf)
arcpy.analysis.Buffer(maui_gravity_mains, mains_buf, main_buffer, dissolve_option="ALL")

# 2) Select parcels that intersect mains buffer; then ADD parcels intersecting raw laterals (no buffer)
parcels_lyr = "maui_parcels_lyr"
arcpy.management.MakeFeatureLayer(maui_parcels, parcels_lyr)

# Start with mains buffer selection
arcpy.management.SelectLayerByLocation(parcels_lyr, "INTERSECT", mains_buf, selection_type="NEW_SELECTION")

# If laterals exist, add parcels that intersect the (unbuffered) laterals lines
if arcpy.Exists(maui_laterals):
    arcpy.management.SelectLayerByLocation(parcels_lyr, "INTERSECT", maui_laterals, selection_type="ADD_TO_SELECTION")

# Save selected parcels
parcels_with_infra = r"in_memory\parcels_with_infra"
if arcpy.Exists(parcels_with_infra):
    arcpy.management.Delete(parcels_with_infra)
arcpy.management.CopyFeatures(parcels_lyr, parcels_with_infra)
arcpy.management.Delete(parcels_lyr)

# 3) Mark OSDS Lat = 'Y' if inside those parcels, else 'N'
if not any(f.name.lower() == "lat" for f in arcpy.ListFields(maui_osds_points)):
    arcpy.management.AddField(maui_osds_points, "Lat", "TEXT", field_length=16)

# Default to 'N'
arcpy.management.CalculateField(maui_osds_points, "Lat", "'N'", "PYTHON3")

# Flip to 'Y' where OSDS intersects selected parcels
osds_lyr = "maui_osds_lyr"
arcpy.management.MakeFeatureLayer(maui_osds_points, osds_lyr)
arcpy.management.SelectLayerByLocation(osds_lyr, "INTERSECT", parcels_with_infra)
selected = int(arcpy.management.GetCount(osds_lyr)[0])
if selected > 0:
    arcpy.management.CalculateField(osds_lyr, "Lat", "'Y'", "PYTHON3")
arcpy.management.Delete(osds_lyr)

print(f"Done. Marked Lat='Y' for {selected} OSDS point(s).")


Done. Marked Lat='Y' for 1119 OSDS point(s).


#### Same for Hawaii County 

In [6]:

hi_osds_points   = r"C:\Users\aimee\Documents\SewerBillingCleanup\Downloaded_data\NEW_OSDS\NewOSDSv2\Hawaii_OSDSv2\Hawaii\Hawaii_OSDS_2023.shp"        # OSDS points (Hawai‘i Island)
hi_parcels       = r"C:\Users\aimee\Documents\SewerBillingCleanup\Projected_data\tmk_state_projected\tmk_state_projected.shp"     # TMK parcels (polygons)
hi_laterals      = r"C:\Users\aimee\Documents\SewerBillingCleanup\Downloaded_data\SewerLines\HawaiiCoLaterals_Project\HawaiiCoLaterals_Project.shp"    # laterals (lines)

laterals_buffer = "2 Meters"  # buffer only the laterals

# sanity check
for p in (hi_osds_points, hi_parcels, hi_laterals):
    if not arcpy.Exists(p):
        raise FileNotFoundError(f"Path not found: {p}")

# 1) Buffer laterals by 2 m (dissolve to one mask)
lats_buf = r"in_memory\hi_lats_buf"
if arcpy.Exists(lats_buf):
    arcpy.management.Delete(lats_buf)
arcpy.analysis.Buffer(hi_laterals, lats_buf, laterals_buffer, dissolve_option="ALL")

# 2) Select parcels that intersect the laterals buffer
parcels_layer = "hi_parcels_lyr"
arcpy.management.MakeFeatureLayer(hi_parcels, parcels_layer)
arcpy.management.SelectLayerByLocation(parcels_layer, "INTERSECT", lats_buf, selection_type="NEW_SELECTION")
parcels_with_infra = r"in_memory\hi_parcels_with_infra"
if arcpy.Exists(parcels_with_infra):
    arcpy.management.Delete(parcels_with_infra)
arcpy.management.CopyFeatures(parcels_layer, parcels_with_infra)
arcpy.management.Delete(parcels_layer)

# 3) Ensure OSDS has 'Lat' (TEXT), set all to 'N', then flip to 'Y' where inside those parcels
if not any(f.name.lower() == "lat" for f in arcpy.ListFields(hi_osds_points)):
    arcpy.management.AddField(hi_osds_points, "Lat", "TEXT", field_length=16)

arcpy.management.CalculateField(hi_osds_points, "Lat", "'N'", "PYTHON3")

osds_layer = "hi_osds_lyr"
arcpy.management.MakeFeatureLayer(hi_osds_points, osds_layer)
arcpy.management.SelectLayerByLocation(osds_layer, "INTERSECT", parcels_with_infra, selection_type="NEW_SELECTION")
selected = int(arcpy.management.GetCount(osds_layer)[0])
if selected > 0:
    arcpy.management.CalculateField(osds_layer, "Lat", "'Y'", "PYTHON3")
arcpy.management.Delete(osds_layer)

print(f"Done. Marked Lat='Y' for {selected} OSDS point(s) on Hawai‘i Island.")


Done. Marked Lat='Y' for 240 OSDS point(s) on Hawai‘i Island.


##### Merge all the OSDS datasets into one shapefile and join the sewer billing data by TMK to populate the SewBill field

In [10]:

# === Uses variables you already have in the notebook ===
# Expecting these to exist from earlier cells; only the ones that exist will be used.
var_names = ["osds_points_path", "maui_osds_points", "kauai_osds_path", "hi_osds_points"]
osds_paths = [globals()[v] for v in var_names if v in globals()]
if not osds_paths:
    raise ValueError("No OSDS path variables found. Define e.g. oahu_osds_points, maui_osds_points, etc.")

# Output merged shapefile (edit if you want a different location/name)
merged_osds = r"C:\Users\aimee\Documents\SewerBillingCleanup\Downloaded_data\NEW_OSDS\All_OSDS_merged\All_OSDS_merged.shp"

# TMK (LONG) field name on your OSDS layers (same across them)
osds_tmk_field = "TMK"   # edit if yours is different

# Billing summary: use OUT_CSV from your earlier stats cell if it exists; otherwise set a path.
billing_summary_path = OUT_CSV if "OUT_CSV" in globals() else r"C:\Users\aimee\Documents\SewerBillingCleanup\Downloaded_data\SewerBillingData\Sum_Mau_Kau_HNL_Billing.csv"

# --- Merge the island OSDS ---
if os.path.exists(merged_osds):
    arcpy.management.Delete(merged_osds)
arcpy.management.Merge(osds_paths, merged_osds)

# Ensure SewBill exists and default to 'N'
if not any(f.name.lower() == "sewbill" for f in arcpy.ListFields(merged_osds)):
    arcpy.management.AddField(merged_osds, "SewBill", "TEXT", field_length=16)
arcpy.management.CalculateField(merged_osds, "SewBill", "'N'", "PYTHON3")

# --- Load billing summary to an OID-backed table (handles CSV) ---
billing_tbl = r"in_memory\billing_summary"
if arcpy.Exists(billing_tbl):
    arcpy.management.Delete(billing_tbl)
arcpy.conversion.TableToTable(billing_summary_path, r"in_memory", os.path.basename(billing_tbl))

# Pick TMK field from the billing table; prefer LONG, else convert TEXT -> LONG
b_fields = {f.name.lower(): f for f in arcpy.ListFields(billing_tbl)}
billing_tmk_field = next((b_fields[n].name for n in ("cleantmk_long","tmk_long","cleantmk","tmk") if n in b_fields), None)
if billing_tmk_field is None:
    raise ValueError("Could not find a TMK field in the billing summary.")

billing_join_field = billing_tmk_field
if b_fields[billing_tmk_field.lower()].type == "String":
    # make numeric join key from digits
    if not any(f.name == "TMK_join" for f in arcpy.ListFields(billing_tbl)):
        arcpy.management.AddField(billing_tbl, "TMK_join", "LONG")
    arcpy.management.CalculateField(
        billing_tbl, "TMK_join", f"to_long(!{billing_tmk_field}!)", "PYTHON3",
        "def to_long(v):\n"
        "    s = '' if v is None else str(v)\n"
        "    d = ''.join(c for c in s if c.isdigit())\n"
        "    return int(d) if d else None\n"
    )
    billing_join_field = "TMK_join"

# Find COUNT* field and normalize to a short stable name 'BillCount'
count_src = next((f.name for f in arcpy.ListFields(billing_tbl) if f.name.upper().startswith("COUNT")), None)
if count_src is None:
    raise ValueError("Could not find a COUNT* field in the billing summary.")
if not any(f.name == "BillCount" for f in arcpy.ListFields(billing_tbl)):
    arcpy.management.AddField(billing_tbl, "BillCount", "LONG")
arcpy.management.CalculateField(billing_tbl, "BillCount", f"!{count_src}!", "PYTHON3")

# Drop any existing BillCount on merged OSDS (from prior runs) to avoid join collisions
if any(f.name == "BillCount" for f in arcpy.ListFields(merged_osds)):
    arcpy.management.DeleteField(merged_osds, ["BillCount"])

# --- Join BillCount to merged OSDS by TMK ---
arcpy.management.JoinField(
    in_data=merged_osds,
    in_field=osds_tmk_field,
    join_table=billing_tbl,
    join_field=billing_join_field,
    fields=["BillCount"]
)

# --- Set SewBill = 'Y' where BillCount > 0 ---
lyr = "all_osds_lyr"
arcpy.management.MakeFeatureLayer(merged_osds, lyr)
bc_delim = arcpy.AddFieldDelimiters(merged_osds, "BillCount")
arcpy.management.SelectLayerByAttribute(lyr, "NEW_SELECTION", f"{bc_delim} > 0")
if int(arcpy.management.GetCount(lyr)[0]) > 0:
    arcpy.management.CalculateField(lyr, "SewBill", "'Y'", "PYTHON3")
arcpy.management.Delete(lyr)

# (optional) clean up the joined BillCount field
arcpy.management.DeleteField(merged_osds, ["BillCount"])

print(f"Merged {len(osds_paths)} OSDS layers into: {merged_osds}\nSewBill populated from billing by TMK.")


Merged 4 OSDS layers into: C:\Users\aimee\Documents\SewerBillingCleanup\Downloaded_data\NEW_OSDS\All_OSDS_merged\All_OSDS_merged.shp
SewBill populated from billing by TMK.


#### Check private wastewater service areas

In [21]:
priv_service_areas = r"C:\Users\aimee\Documents\SewerBillingCleanup\Downloaded_data\Priority_WWTP_Service_Areas_DRAFT\Priority_WWTP_Service_Areas.shp"  # polygons

# default all to 'N'
arcpy.management.CalculateField(merged_osds, "Priv", "'N'", "PYTHON3")

# select OSDS that intersect private service area polygons
osds_lyr = "merged_osds_lyr"
arcpy.management.MakeFeatureLayer(merged_osds, osds_lyr)
arcpy.management.SelectLayerByLocation(osds_lyr, "INTERSECT", priv_service_areas, selection_type="NEW_SELECTION")
selected = int(arcpy.management.GetCount(osds_lyr)[0])

# set Priv = 'Y' for selected
if selected > 0:
    arcpy.management.CalculateField(osds_lyr, "Priv", "'Y'", "PYTHON3")
arcpy.management.Delete(osds_lyr)

print(f"Priv set to 'Y' for {selected} OSDS feature(s); all others 'N'.")

Priv set to 'Y' for 1451 OSDS feature(s); all others 'N'.


In [15]:
from collections import Counter

# === EDIT THIS if you didn't keep the same variable name ===
merged_osds_path = r"C:\Users\aimee\Documents\SewerBillingCleanup\Downloaded_data\NEW_OSDS\All_OSDS_merged\All_OSDS_merged.shp"  # or r"C:\path\to\All_OSDS_merged.shp"

# What fields do we summarize?
flag_fields = ["Lat", "SewBill", "Priv"]

# Detect which fields actually exist
existing = {f.name for f in arcpy.ListFields(merged_osds_path)}
flags = [f for f in flag_fields if f in existing]

# Try to find an island field (optional)
island_field = None
for cand in ("ISLAND", "Island", "island"):
    if cand in existing:
        island_field = cand
        break

# Build a search cursor
fields = [island_field] * (island_field is not None) + flags  # e.g., ['ISLAND','Lat','SewBill','Priv'] or just flags
with arcpy.da.SearchCursor(merged_osds_path, fields) as cur:
    total = 0
    # overall counts for each flag
    counts = {f: Counter() for f in flags}
    # by-island counts
    by_island = {}
    # combo counts (e.g., Lat/SewBill/Priv tuple)
    combo = Counter()

    for row in cur:
        total += 1
        # unpack row
        if island_field:
            island = row[0]
            vals = row[1:]
        else:
            island = None
            vals = row

        # update flag tallies
        for i, f in enumerate(flags):
            val = (vals[i] or "").upper()
            counts[f][val] += 1

        # by island
        if island_field:
            d = by_island.setdefault(island, {f: Counter() for f in flags})
            for i, f in enumerate(flags):
                val = (vals[i] or "").upper()
                d[f][val] += 1

        # combo (keep the order of flags)
        combo[tuple((vals[i] or "").upper() for i in range(len(flags)))] += 1

# ---- Pretty print (no pandas required) ----
def pct(n):  # tiny helper
    return f"{(100*n/total):.1f}%" if total else "0.0%"

print(f"Total OSDS features: {total}\n")

# Overall flag tables
for f in flags:
    y = counts[f].get("Y", 0)
    n = counts[f].get("N", 0)
    other = total - y - n
    print(f"{f}:  Y={y} ({pct(y)})   N={n} ({pct(n)})   other/blank={other} ({pct(other)})")
print()

# By-island summary (if available)
if island_field:
    print(f"By {island_field} summary:")
    for island, d in by_island.items():
        line = [f"{island}:"]
        for f in flags:
            y = d[f].get("Y", 0)
            n = d[f].get("N", 0)
            line.append(f"{f}[Y={y}, N={n}]")
        print("  " + "  ".join(line))
    print()

# Combo table
print("Combination counts (ordered as: " + ", ".join(flags) + "):")
for key, c in combo.most_common():
    print(f"  {key}: {c}")


Total OSDS features: 103529

Lat:  Y=1938 (1.9%)   N=101591 (98.1%)   other/blank=0 (0.0%)
SewBill:  Y=1798 (1.7%)   N=101731 (98.3%)   other/blank=0 (0.0%)
Priv:  Y=1451 (1.4%)   N=102078 (98.6%)   other/blank=0 (0.0%)

By Island summary:
  Oahu:  Lat[Y=558, N=13422]  SewBill[Y=481, N=13499]  Priv[Y=290, N=13690]
  Maui:  Lat[Y=1110, N=12285]  SewBill[Y=583, N=12812]  Priv[Y=182, N=13213]
  Molokai:  Lat[Y=6, N=1687]  SewBill[Y=0, N=1693]  Priv[Y=1, N=1692]
  Lanai:  Lat[Y=3, N=4]  SewBill[Y=0, N=7]  Priv[Y=0, N=7]
  Kauai:  Lat[Y=21, N=14494]  SewBill[Y=734, N=13781]  Priv[Y=578, N=13937]
  Hawaii:  Lat[Y=239, N=59669]  SewBill[Y=0, N=59908]  Priv[Y=399, N=59509]
   :  Lat[Y=1, N=30]  SewBill[Y=0, N=31]  Priv[Y=1, N=30]

Combination counts (ordered as: Lat, SewBill, Priv):
  ('N', 'N', 'N'): 99063
  ('N', 'N', 'Y'): 1405
  ('Y', 'N', 'N'): 1222
  ('N', 'Y', 'N'): 1119
  ('Y', 'Y', 'N'): 674
  ('Y', 'N', 'Y'): 41
  ('N', 'Y', 'Y'): 4
  ('Y', 'Y', 'Y'): 1


In [18]:
merged_osds_path = merged_osds  # or set to r"C:\path\to\All_OSDS_merged.shp"
island_field     = "ISLAND"         # island name field in merged OSDS
island_names     = {                # match the exact values in your data
    "Kauai":  "Kauai",
    "Oahu":   "Oahu",
    "Maui":   "Maui",
    "Hawaii": "Hawaii"              # change to "Hawaiʻi" if that’s how it appears
}

# Default everyone to 'N'
arcpy.management.CalculateField(merged_osds_path, "Flag", "'N'", "PYTHON3")

# Make a layer and build delimited field names for SQL
lyr = "osds_flag_lyr"
arcpy.management.MakeFeatureLayer(merged_osds_path, lyr)

ISL  = arcpy.AddFieldDelimiters(merged_osds_path, island_field)
LAT  = arcpy.AddFieldDelimiters(merged_osds_path, "Lat")
SEWB = arcpy.AddFieldDelimiters(merged_osds_path, "SewBill")
PRIV = arcpy.AddFieldDelimiters(merged_osds_path, "Priv")

def apply_rule(island_value, where_condition):
    """Select rows for one island that meet condition; set Flag='Y'."""
    where = f"{ISL} = '{island_value}' AND ({where_condition})"
    arcpy.management.SelectLayerByAttribute(lyr, "NEW_SELECTION", where)
    n = int(arcpy.management.GetCount(lyr)[0])
    if n > 0:
        arcpy.management.CalculateField(lyr, "Flag", "'Y'", "PYTHON3")
    return n

# Rules:
# Kauai: Lat OR SewBill OR Priv
n_kauai = apply_rule(island_names["Kauai"],  f"{LAT} = 'Y' OR {SEWB} = 'Y' OR {PRIV} = 'Y'")
# Oahu:  Lat OR Priv
n_oahu  = apply_rule(island_names["Oahu"],   f"{LAT} = 'Y' OR {PRIV} = 'Y'")
# Maui:  Lat OR SewBill OR Priv
n_maui  = apply_rule(island_names["Maui"],   f"{LAT} = 'Y' OR {SEWB} = 'Y' OR {PRIV} = 'Y'")
# Hawai‘i: Lat OR Priv
n_hawaii= apply_rule(island_names["Hawaii"], f"{LAT} = 'Y' OR {PRIV} = 'Y'")

arcpy.management.Delete(lyr)
print(f"Flag='Y' counts — Kauai:{n_kauai}  Oahu:{n_oahu}  Maui:{n_maui}  Hawaii:{n_hawaii}")

Flag='Y' counts — Kauai:1331  Oahu:846  Maui:1313  Hawaii:637
